# Imports

In [ ]:
import joblib
import os
from skimage import data, color
from skimage.transform import resize
import numpy as np
import importlib

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
#import sys
#sys.path.append('/content/drive/MyDrive/machine-learning/obligatorio/detector')

In [ ]:
import utils
importlib.reload(utils)
from utils import (
    grid_search,
    plot_f1_heatmap,
    plot_top_combinations_bar,
    show_image
)

#BASE_PATH = '/content/drive/MyDrive/machine-learning/obligatorio/detector'
BASE_PATH = ''

BATCH_SIZE = 1024
MAX_JOBS = 4

MODEL_NAME = 'model_x2_v2_50'
SCALER_NAME = 'scaler_x2_v2_50'
PCA_NAME = 'pca_x2_v2_50'

# Clasificador

In [ ]:
# Cargar modelo entrenado
clf = joblib.load(os.path.join(BASE_PATH, f'models/{MODEL_NAME}.joblib'))

# Cargar standard scaler
scaler = joblib.load(os.path.join(BASE_PATH, f'models/{SCALER_NAME}.joblib'))

# Cargar pca
pca = joblib.load(os.path.join(BASE_PATH, f'models/{PCA_NAME}.joblib'))

# Cargar astronauta

In [ ]:
dataset = []
astronaut = data.astronaut()
astronaut = color.rgb2gray(astronaut)
astronaut = resize(astronaut, (500, 500), anti_aliasing=True, preserve_range=True)
astronaut = (astronaut * 255).astype(np.uint8)
# (filename, image, true_faces)
dataset.append(('astronaut', astronaut, 1))
show_image(astronaut, cmap='gray', figsize=(4, 4))

# Grid Search

In [ ]:
# Hiperparámetros a evaluar
thresholds = [0.7, 0.8, 0.9]
steps = [2, 4]
overlaps = [0.05, 0.2, 0.3]
test_scales = np.arange(0.5, 5.0, 0.5)

In [ ]:
df_grid, best_config = grid_search(
    clf,
    scaler,
    pca,
    dataset,
    thresholds=thresholds,
    steps=steps,
    overlaps=overlaps,
    test_scales=test_scales,
    batch_size=BATCH_SIZE,
    max_jobs=MAX_JOBS,
    base_dir=BASE_PATH,
    n_images_max=1,
    early_stop_f1=0.4
)
plot_f1_heatmap(df_grid, fix_overlap=0.2)
plot_top_combinations_bar(df_grid, top_n=5)

In [ ]:
# save results
save_dir = os.path.join(BASE_PATH, "results")
df_grid.to_csv(f"{save_dir}/grid_search.csv", index=False)
print(f"Resultados guardados en {save_dir}/grid_search.csv")